# Using EcoFOCIpy to process raw field data

## SKQ202312S (Spring Mooring Cruise - Oscar Dyson)

## BO Oxygen Data



In [1]:
import yaml
import glob
import pandas as pd
import os

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

ModuleNotFoundError: No module named 'EcoFOCIpy'

In [ ]:
sample_data_dir = '/Users/bell/ecoraid/2023/CTDcasts/skq202312s/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [ ]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
oxydatafile = sample_data_dir+'working/DiscreteOxygens/SKQ2023-12 Oxygen Data.txt' #<- point to cruise and process all files within
cruise_name = 'skq202312s' #no hyphens
cruise_meta_file = sample_data_dir+'logs/SKQ202312S.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpOxy.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

## Load csv Oxygen File

In [ ]:
oxy_data = pd.read_csv(oxydatafile,delimiter='\t')
oxy_data

In [ ]:
oxy_data.dropna(how='all', inplace=True)

## Merge Bottle and Nutrient Data but drop non nutrient vars?

In [ ]:
keep_param = ['bottle','prdm']

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(oxy_data[oxy_data['Cast']==matchcast],cdata.reset_index()[keep_param],right_on='bottle',left_on='Niskin').set_index('bottle').drop(columns=['Cast'])
        cruise_data[cast] = cruise_data[cast].dropna(how='all', axis=1)
    except:
        continue

## Add Deployment meta information

In [ ]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [ ]:
with open(inst_meta_file) as file:
    inst_oxy_config = yaml.full_load(file)

## Add institutional meta-information


In [ ]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [ ]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'O2 (uM)':'O2',
                            'O2, uM/l':'O2',
                            'Oxygen (uM)':'O2',
                            'Niskin':'BTLID',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_oxy_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.lower().split('d')[-1].split('.')[0].zfill(3)+'_oxy.nc',format="NETCDF3_CLASSIC")
    except KeyError:
        print(f'Skipping {cast}')
    except RuntimeError:
        print(f'Skipping & Removing {cast}')
        os.remove(path=cruise_name+'c'+cast.lower().split('d')[-1].split('.')[0].zfill(3)+'_oxy.nc')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history